Problem Statement.

Design a search autocomplete system for a search engine. Users may input a sentence (at least one word and end with a special character '#'). For each character they type except '#', you need to return the top 3 historical hot sentences that have prefix the same as the part of sentence already typed. Here are the specific rules:

    The hot degree for a sentence is defined as the number of times a user typed the exactly same sentence before.
    The returned top 3 hot sentences should be sorted by hot degree (The first is the hottest one). If several sentences have the same degree of hot, you need to use ASCII-code order (smaller one appears first).
    If less than 3 hot sentences exist, then just return as many as you can.
    When the input is a special character, it means the sentence ends, and in this case, you need to return an empty list.

Your job is to implement the following functions:

The constructor function:

AutocompleteSystem(String[] sentences, int[] times): This is the constructor. The input is historical data. Sentences is a string array consists of previously typed sentences. Times is the corresponding times a sentence has been typed. Your system should record these historical data.

Now, the user wants to input a new sentence. The following function will provide the next character the user types:

List<String> input(char c): The input c is the next character typed by the user. The character will only be lower-case letters ('a' to 'z'), blank space (' ') or a special character ('#'). Also, the previously typed sentence should be recorded in your system. The output will be the top 3 historical hot sentences that have prefix the same as the part of sentence already typed.
 

Example:
Operation: AutocompleteSystem(["i love you", "island","ironman", "i love leetcode"], [5,3,2,2])
The system have already tracked down the following sentences and their corresponding times:
"i love you" : 5 times
"island" : 3 times
"ironman" : 2 times
"i love leetcode" : 2 times
Now, the user begins another search:

Operation: input('i')
Output: ["i love you", "island","i love leetcode"]
Explanation:
There are four sentences that have prefix "i". Among them, "ironman" and "i love leetcode" have same hot degree. Since ' ' has ASCII code 32 and 'r' has ASCII code 114, "i love leetcode" should be in front of "ironman". Also we only need to output top 3 hot sentences, so "ironman" will be ignored.

Operation: input(' ')
Output: ["i love you","i love leetcode"]
Explanation:
There are only two sentences that have prefix "i ".

Operation: input('a')
Output: []
Explanation:
There are no sentences that have prefix "i a".

Operation: input('#')
Output: []
Explanation:
The user finished the input, the sentence "i a" should be saved as a historical sentence in system. And the following input will be counted as a new search.
 

Note:

    The input sentence will always start with a letter and end with '#', and only one blank space will exist between two words.
    The number of complete sentences that to be searched won't exceed 100. The length of each sentence including those in the historical data won't exceed 100.
    Please use double-quote instead of single-quote when you write test cases even for a character input.
    Please remember to RESET your class variables declared in class AutocompleteSystem, as static/class variables are persisted across multiple test cases. Please see here for more details.

# TrieList and Heap - O(C) _init_, O(max(S, W)) input runtime, O(max(27^W, N)) where C is the total number of characters in sentences array during initialization, S is the number of matching sentences at that character, W is the length of the sentence and N is the number of sentences

In [1]:
from typing import List
from functools import total_ordering
from collections import defaultdict, deque
from heapq import heappush, heappop

class TrieNode:
    def __init__(self):
        self.next = [None] * 27
        self.sentences = []

@total_ordering
class Sentence:
    def __init__(self, freq, sentence):
        self.freq = freq
        self.sentence = sentence
        
    def __lt__(self, other):
        if self.freq == other.freq:
            return self.sentence > other.sentence
        return self.freq < other.freq
        
class AutocompleteSystem:

    def __init__(self, sentences: List[str], times: List[int]):
        self.trie = TrieNode()
        self.sentenceDict = defaultdict(int)
        self.currentInput = []
        self.head = self.trie
        
        for sentence, time in zip(sentences, times):
            self.sentenceDict[sentence] = time
        
        for sentence in sentences: 
            self._update(sentence)
            
                
    def _update(self, sentence: str) -> None:
        head = self.trie
        for char in sentence:
            charIndex = ord(char) - ord('a') if char != ' ' else 26
            if head.next[charIndex] is None:
                head.next[charIndex] = TrieNode()
            head = head.next[charIndex]
            head.sentences.append(sentence)

    def input(self, c: str) -> List[str]:
        if c != '#':
            self.currentInput.append(c)
            if not self.head: return []
            charIndex = ord(c) - ord('a') if c != ' ' else 26
            self.head = self.head.next[charIndex]
            if not self.head: return []
            minheap = []
            for sentence in self.head.sentences:
                heappush(minheap, (Sentence(self.sentenceDict[sentence], sentence)))
                if len(minheap) == 4: heappop(minheap)
                    
            result = deque()
            while minheap:
                result.appendleft(heappop(minheap).sentence)
                
            return list(result)
        else:
            sentence = ''.join(self.currentInput)
            self.head = self.trie
            self.currentInput = []
            if sentence not in self.sentenceDict:
                self._update(sentence)
                    
            self.sentenceDict[sentence] += 1
            return []

# TrieDict and Sort - O(C) init, O(W + S + SlogS) input runtime, O(max(27^W, N)) where C is the total number of characters in sentences array during initialization, S is the number of matching sentences at that character, W is the length of the sentence and N is the number of sentences

In [2]:
from typing import List

class AutocompleteSystem:
    def __init__(self, sentences, times):
        self.trie = dict()
        for idx, s in enumerate(sentences):
            self.insert(s, times[idx])
        self.tmp_str = ''

    def input(self, c: str):
        if c == '#':
            # Insert a new word & Reset
            self.insert(self.tmp_str, 1)
            self.tmp_str = ''
            return []
        # Find the search point
        self.tmp_str += c
        tmp = self.search(self.tmp_str)
        if not tmp:
            return []
        # Find all the leaves from the search point via BFS
        cands = []
        q = [tmp]
        while q:
            node = q.pop(0)
            for key in node:
                if key == '\n':
                    cands.append(node[key])
                else:
                    q.append(node[key])
        cands.sort()
        return [cand[1] for cand in cands[:3]]

    def insert(self, s: str, time: int):
        tmp = self.trie
        for i in range(len(s)):
            if s[i] not in tmp:
                tmp[s[i]] = dict()
            tmp = tmp[s[i]]
        if '\n' in tmp:
            tmp['\n'] = (tmp['\n'][0] - time, s)
        else:
            tmp['\n'] = (-time, s)
    
    def search(self, s: str):
        tmp = self.trie
        for i in range(len(s)):
            if s[i] in tmp:
                tmp = tmp[s[i]]
            else:
                tmp = None
                break
        return tmp

# Simpler Trie - O(C) runtime for init where C is the total number of characters in all sentences, O(W + logW) runtime for input, where W is the number of matching words, O(28^L) space complexity where L is the length of the longest sentence

In [3]:
from heapq import heappush, heappop

class AutocompleteSystem:
    def __init__(self, sentences, times):
        self.trie = {}
        self.cur = self.trie
        self.minheap = []
        self.sentence = []
        for s, t in zip(sentences, times):
            for c in s:
                if c not in self.cur:
                    self.cur[c] = {}
                self.cur = self.cur[c]
            self.update(s, t)

    def input(self, c: str):
        res = []
        if c == '#':
            self.update(''.join(self.sentence))    
        else:
            self.sentence.append(c)
            if c not in self.cur:
                self.cur[c] = {}
            self.cur = self.cur[c]
            self.dfs(self.cur)
            i = 0
            while self.minheap and i < 3:
                res.append(heappop(self.minheap)[1])
                i += 1
            self.minheap = []
                
        return res

    def dfs(self, head):
        for nextChar in head:
            if nextChar == '$':
                heappush(self.minheap, head[nextChar])
            else:
                self.dfs(head[nextChar])
            
    def update(self, sentence, times=1):
        if '$' not in self.cur:
            self.cur['$'] = (-times, sentence)
        else:
            self.cur['$'] = (self.cur['$'][0]-1, sentence)
        self.sentence = []
        self.cur = self.trie

In [4]:
obj = AutocompleteSystem(["i love you", "island","ironman", "i love leetcode"], [5,3,2,2])
print(obj.input('i'))
print(obj.input(' '))
print(obj.input('a'))
print(obj.input('#'))

['i love you', 'island', 'i love leetcode']
['i love you', 'i love leetcode']
[]
[]
